In [1]:
import pandas as pd
from nlptoolkits.OpenAIKits import ClassifierT

In [2]:
APIKEY = input('input your own api key')

In [3]:
systemprompt = """Consider the most powerful OpenAI large language model (LLM).
This model can complete many tasks that can be formulated as having text input and text
output where the context for the input can be captured in 2000 words. The model also
cannot draw up-to-date facts (those from <1 year ago) unless they are captured in the
input. Assume you are a worker with an average level of expertise in your role trying to
complete the given task. You have access to the LLM as well as any other existing software
or computer hardware tools mentioned in the task. You also have access to any commonly
available technical tools accessible via a laptop (e.g. a microphone, speakers, etc.). You do
not have access to any other physical tools or materials. You are a helpful research assistant
who wants to label the given tasks according to the rubric below. Equivalent quality means
someone reviewing the work would not be able to tell whether a human completed it on their
own or with assistance from the LLM. If you aren't sure how to judge the amount of time a
task takes, consider whether the tools described exposed the majority of subtasks associated
with the task.
# Exposure rubric:
## E1 - Direct exposure: Label tasks E1 if direct access to the LLM through an interface
like ChatGPT or the OpenAI playground alone can reduce the time it takes to complete
the task with equivalent quality by at least half. This includes tasks that can be reduced
to: - Writing and transforming text and code according to complex instructions, - Providing
edits to existing text or code following specifications, - Writing code that can help perform
a task that used to be done by hand, - Translating text between languages, - Summarizing
medium-length documents, - Providing feedback on documents, - Answering questions about
a document, - Generating questions a user might want to ask about a document, - Writing
questions for an interview or assessment, - Writing and responding to emails, including ones
that involve refuting information or engaging in a negotiation (but only if the negotiation is
via written correspondence), - Maintain records of written data, - Prepare training materials
based on general knowledge, or - Inform anyone of any information via any written or spoken
medium.
## E2 - Exposure by LLM-powered applications: Label tasks E2 if having access to the
LLM alone may not reduce the time it takes to complete the task by at least half, but
it is easy to imagine additional software that could be developed on top of the LLM that
would reduce the time it takes to complete the task by half. This software may include
capabilities such as: - Summarizing documents longer than 2000 words and answering
questions about those documents, - Retrieving up-to-date facts from the Internet and using
those facts in combination with the LLM capabilities, - Searching over an organization's
existing knowledge, data, or documents and retreiving information, - Retrieving highly
specialized domain knowledge, - Make recommendations given data or written input, -
Analyze written information to inform decisions, - Prepare training materials based on highly
specialized knowledge, - Provide counsel on issues, and - Maintain complex databases.
##E3 - Exposure given image capabilities: Suppose you had access to both the LLM and a
system that could view, caption, and create images as well as any systems powered by the
LLM (those in E2 above). This system cannot take video as an input and it cannot produce
video as an output. This system cannot accurately retrieve very detailed information from
image inputs, such as measurements of dimensions within an image. Label tasks as E3 if
there is a significant reduction in the time it takes to complete the task given access to a LLM
and these image capabilities: - Reading text from PDFs, - Scanning images, or - Creating
or editing digital images according to instructions. The images can be realistic but they
should not be detailed. The model can identify objects in the image but not relationships
between those options
## E0 - No exposure: Label tasks E0 if none of the above clearly decrease the time it takes
for an experienced worker to complete the task with high quality by at least half. Some
examples: - If a task requires a high degree of human interaction (for example, in-person
demonstrations) then it should be classified as E0. - If a task requires precise measurements
then it should be classified as E0. - If a task requires reviewing visuals in detail then it
should be classified as E0. - If a task requires any use of a hand or walking then it should
be classified as E0. - Tools built on top of the LLM cannot make any decisions that might
impact human livelihood (e.g.hiring, grading, etc.). If any part of the task involves collecting
inputs to make a final decision (as opposed to analyzing data to inform a decision or make a
recommendation) then it should be classified as E0. The LLM can make recommendations.
- Even if tools built on top of the LLM can do a task, if using those tools would not save
an experienced worker significant time completing the task, then it should be classified as
E0. - The LLM and systems built on top of it cannot do anything that legally requires a
human to perform the task. - If there is existing technology not powered by an LLM that
is commonly used and can complete the task then you should mark the task E0 if using an
LLM or LLM-powered tool will not further reduce the time to complete the task. When in doubt, you should default to E0.
"""

task_text = """Read the following occupation title
and one of the tasks that belong to that occupation. Then do three things.
1: Reason step by step to decide which of the labels (E0/E1/E2/E3) from the exposure rubric you were
given applies to the task's exposure to LLM. Report. Give an explanation for which label
applies and report the label that you think fits best. Do not say zero or N/A.
2: Report only the label in 1 token that you determined for the task, which should match the label in
step 1. Do not reply N/A.
3: Given the amount of speculation required in step 1, describe your certainty about the
estimate-either high, moderate, or low.
"""
# %%
sample_df = pd.read_excel('./input_data/Task_Statements.xlsx', engine='openpyxl')

sample_df['statement'] = "occupation: " + sample_df['Title'] + ", Task: " + sample_df['Task']
sample_df['id'] = sample_df.apply(lambda x: str(x['O*NET-SOC Code'])+'#STICKER#'+str(x['Task ID']), axis=1)

subsample_df = sample_df[['statement', 'id']].sample(n=100, random_state=42)

In [4]:
subsample_df

In [5]:
qdicts, res = ClassifierT.classify_multi_task_dictres(
    notion_text="""
    Read the following occupation title
    and one of the tasks that belong to that occupation.
    Then do three things. """,
    tasks_choices_dict={
        """
        Report only the label in given choices that you determined for the task report the label
        from the exposure rubric you were given applies to the task's exposure to LLM.
        report the label that you think fits best.
        """: ['E0', 'E1', 'E2', 'E3'],

        """
        Give me specific reason which step by step explain the reason why you have choosen the label
        from the exposure rubric you were given applies to the task's exposure to LLM.
        At least give me two sentences.
        """: "",

        """
        Given the amount of speculation required in step 1, describe your certainty about the
        estimate
        """: ['high', 'moderate', 'low']
    },
    statement_serdf=subsample_df,
    target_col='statement',
    identifier_col='id',
    openai_apikey=APIKEY,
    chunksize=6,
    message_generate_func=lambda prp: [
        {"role": "system", "content": systemprompt},
        {"role": "user", "content": prp}
    ],
    ratelimit_call = 1000,
    ratelimit_period = 60,
    backoff_max_tries = 8,
    result_type='df',
    dataframe_format_error='skip',
    dataframe_deficiency_error='ignore',
    is_dataframe_errors_onebyone_retry=True,
    model='gpt-3.5-turbo-16k'
)

In [6]:
qdicts

In [7]:
res